In [9]:
import pandas as pd
import cv2
import numpy as np
import os

<img src='ex/1.png' width=600, heigth=540>

In [300]:
N = ord('K') % 5 + 1

<img src='ex/2-3.png' width=600, heigth=540>

In [302]:
filename = 'data/kr1.xlsx'
df = pd.read_excel(filename)

file_name = df.loc[N - 1, 'file name']
image_size = df.loc[N - 1, 'image size']
glasses_color = df.loc[N - 1, 'glasses color']
line_width = int(df.loc[N - 1, 'line width'])

print(f'File: {file_name}, size: {image_size}, color: {glasses_color}, width: {line_width}')

File: emma-watson2.jpg, size: 300x300, color: red, width: 2


<img src='ex/4.png' width=600, heigth=540>

In [257]:
color_dict = {
    'cyan': (0, 255, 255),
    'magenta': (255, 0, 255),
    'yellow': (255, 255, 0),
    'red': (255, 0, 0),
    'blue': (0, 0, 255),
    'green': (0, 255, 0)
}

def get_color(color_name):
    return color_dict.get(color_name.lower(), (0, 0, 0))

def get_size(size_str):
    try:
        width, height = map(int, size_str.lower().split('x'))
        return (width, height)
    except ValueError:
        raise ValueError("Невірний формат розміру, спробуйте '300x300'.")

def add_glasses(image_path, output_size, glasses_color, line_width=3):

    output_size = get_size(output_size)
    
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Завантаження Haar Cascade для обличчя та очей
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    
    # Знаходження облич на зображенні
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))
    
    if len(faces) == 0:
        print("Обличчя не знайдено")
        return
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = image[y:y + h, x:x + w]
        
        eyes = eye_cascade.detectMultiScale(roi_gray)
        if len(eyes) < 2:
            print("Не вдалося знайти два ока")
            continue
        if(len(eyes) != 2):
            eyes = sorted(eyes, key=lambda eye: (eye[1], eye[0]))
        else:
            eyes = sorted(eyes, key=lambda eye: (eye[0], eye[1]))
        
        eye1, eye2 = eyes[:2]
        
        eye1_center = (x + eye1[0] + eye1[2] // 2, y + eye1[1] + eye1[3] // 2)
        eye2_center = (x + eye2[0] + eye2[2] // 2, y + eye2[1] + eye2[3] // 2)
        
        eye_distance = abs(eye2_center[0] - eye1_center[0])
        radius = int(eye_distance * 0.3)
        
        color = get_color(glasses_color)
        
        # Малювання круглих окулярів
        cv2.circle(image, eye1_center, radius, color, line_width)
        cv2.circle(image, eye2_center, radius, color, line_width)
        
        # Додавання перенісся
        bridge_start = (eye1_center[0] + radius, eye1_center[1])
        bridge_end = (eye2_center[0] - radius, eye2_center[1])
        cv2.line(image, bridge_start, bridge_end, color, line_width)
        
        # Додавання дужок окулярів
        midpoint = ((eye1_center[0] + eye2_center[0]) // 2, (eye1_center[1] + eye2_center[1]) // 2)
        
        # Довжина дужок
        hook_length = int(1.75 * radius)
        ear_hook_left_end = (eye1_center[0] - hook_length, eye1_center[1] - radius // 2)
        ear_hook_right_end = (eye2_center[0] + hook_length, eye2_center[1] - radius // 2)
        
        cv2.line(image, (eye1_center[0] - radius, eye1_center[1]), ear_hook_left_end, color, line_width)
        cv2.line(image, (eye2_center[0] + radius, eye2_center[1]), ear_hook_right_end, color, line_width)

    face_region = image[y:y + h, x:x + w]
    resized_face = cv2.resize(face_region, output_size, interpolation=cv2.INTER_AREA)
    
    output_path = f'output/output_{os.path.basename(image_path)}'
    cv2.imwrite(output_path, resized_face)
    print(f"Зображення збережено як {output_path}")
    cv2.imshow('img',resized_face)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [296]:
path = f'data/images/{file_name}'
add_glasses(path, image_size, glasses_color, line_width)

Зображення збережено як output/output_ron_wesley.jpg
